## Linear Regression

In [32]:
# Get user input for the week number to predict
num_week = int(input("Enter the week to predict: "))

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

positions = ["qb", "rb", "wr", "te"]
positions = ["qb"]


for pos in positions:
    # Load your dataset
    data = pd.read_csv("datasets/weekly_scoring.csv")

    # Preprocessing
    data = data[data['POS'] == pos]
    weights = data['WEIGHT']
    
    if(pos == 'qb'):
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING LG', 'RECEIVING 20+', 'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG',
        'RUSHING 20+', 'DATE', 'YEAR', 'WEIGHT', 'TEAM', 'OPP'])
            # Define the list of variables to predict :)
        var_list = ['PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']
    if(pos == 'rb'):
        # Define the list of variables to drop and predict
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        'PASSING Y/A', 'PASSING TD', 'PASSING INT', 'RECEIVING LG', 'RECEIVING 20+',
        'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG',
        'RUSHING 20+', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']
    if(pos == 'wr'):
        # Define the list of variables to drop and predict
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RECEIVING LG', 'RECEIVING 20+',
        'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']
    if(pos == 'te'):
        # Define the list of variables to drop and predict
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RECEIVING LG', 'RECEIVING 20+',
        'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']

    data = pd.get_dummies(data, columns=['PLAYER'], drop_first=True)

    # Identify columns with missing values before imputation
    columns_with_missing = data.columns[data.isnull().any()].tolist()

    # Impute missing values with the mean of each column
    imputer = SimpleImputer(strategy='mean')
    data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)



    # Separate the dataset into features (X) and the target variable (y)
    X = data.drop(var_list, axis=1)

    y = data['MISC FPTS']

    # Save a copy of the dataset
    df = data

    # Instantiate the Linear Regression model without hyperparameter tuning
    lr_model = LinearRegression(fit_intercept=False)

    # Fit the model directly without GridSearchCV
    lr_model.fit(X, y)

    print(lr_model.intercept_, lr_model.coef_, lr_model.score(X, y))

    # # Get a list of unique player names after one-hot encoding
    unique_players = X.columns

    # Create a list of dictionaries to store the results
    results_list = []

    for player in unique_players:
        # Create a DataFrame with all zeros
        week6_data = pd.DataFrame(0, index=range(1), columns=X.columns)
        # Set the corresponding player's column to 1 for prediction
        week6_data[player] = 1
        # Make a prediction for the player
        misc_fpts_prediction = lr_model.predict(week6_data)
        results_list.append({'Player': player, 'MISC FPTS': misc_fpts_prediction[0]})

    # Convert the list of dictionaries into a DataFrame
    results_df = pd.DataFrame(results_list)

    results_df = results_df.sort_values(by='MISC FPTS', ascending=False)

    # # Save the results to a CSV file
    file_name = f"predictions/LRweek{num_week}{pos}unweighted.csv"
    results_df.to_csv(file_name, index=False)

0.0 [ 1.36651163e+01 -0.00000000e+00  3.46000000e+00 -0.00000000e+00
  9.37500000e+00  3.75000000e+00  7.36000000e+00  3.20714286e+00
  1.32555556e+01 -0.00000000e+00 -0.00000000e+00  1.42500000e+01
  3.64705882e-01  9.12500000e-01 -0.00000000e+00  1.85333333e+00
 -0.00000000e+00  1.30000000e+00  1.45842105e+01  3.50000000e+00
  9.89000000e+00  1.18750000e-01  1.77700000e+01  1.14125000e+01
  1.10485714e+01  1.75882353e+00 -0.00000000e+00  4.00000000e-01
  2.07500000e+00 -0.00000000e+00  3.66666667e+00  7.40000000e-01
  6.50000000e+00  3.97272727e+00  1.91631579e+01  1.43378378e+01
  1.63076923e+00 -0.00000000e+00  9.56842105e+00  1.02500000e+01
  1.50000000e+01  1.13375000e+01  1.10571429e+01  4.60000000e-01
  3.52500000e+00 -0.00000000e+00 -0.00000000e+00 -1.90476190e-02
  9.87222222e+00  7.80000000e+00  1.60806452e+01 -0.00000000e+00
 -0.00000000e+00  4.00000000e+00  1.16666667e-01  7.17428571e+00
 -1.00000000e-02  1.06923077e+00 -0.00000000e+00 -0.00000000e+00
  2.29725000e+01  9.1